# Anotação aos professores e avaliadores:
Para o funcionamento adequado do código, é necessário que a variável `AZURE_ACCOUNT_KEY` na célula abaixo seja preenchida com a **secret key** enviada junto ao PDF do projeto, ou que essa chave seja configurada como variável de ambiente `export AZURE_ACCOUNT_KEY="secret_key_aqui"`.
Essa medida foi adotada por questões de segurança, uma vez que o repositório é público.
Agradecemos desde já pela compreensão e colaboração.

In [ ]:
# Coloque aqui sua chave de conta do Azure
AZURE_ACCOUNT_KEY = ""  # <-- INSIRA AQUI SUA CHAVE DO AZURE Entre as aspas duplas

# Pipeline de Tratamento & Upload para Azure



> Caso instalar dependências seja necessário, use a célula abaixo. **Opcional**.


In [11]:

# Instalação de dependências (execute se necessário)
# - azure-storage-blob
# - pandas (opcional, caso use o bloco de tratamento)
#
# Dica: em ambientes corporativos, prefira instalar via conda/anaconda.
# !pip install --quiet azure-storage-blob pandas


## Imports

In [12]:
import os
from datetime import datetime
from io import BytesIO, StringIO

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import ResourceExistsError
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

from tratamento import *

## Configuração e Verificação de arquivos CSV

In [13]:
AZURE_ACCOUNT_KEY = os.environ.get("AZURE_ACCOUNT_KEY", AZURE_ACCOUNT_KEY)
if not AZURE_ACCOUNT_KEY:
    raise ValueError("A chave da conta do Azure não foi definida. Por favor, defina a variável AZURE_ACCOUNT_KEY.")

AZURE_CONNECTION_STRING = f"DefaultEndpointsProtocol=https;AccountName=totsvisiodb;AccountKey={AZURE_ACCOUNT_KEY};EndpointSuffix=core.windows.net"

# Inicializa o cliente do Blob
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)

In [14]:
CONTAINER_NAME = "csvs-brutos-totvs"
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

In [15]:
# Listar arquivos
print("📂 Listando arquivos presentes no container:")
blobs = list(container_client.list_blobs())

if blobs:
    for blob in blobs:
        print(f" - {blob.name}")
else:
    print("⚠ Nenhum arquivo encontrado no container.")


📂 Listando arquivos presentes no container:
 - clientes_desde.csv
 - contratacoes_ultimos_12_meses.csv
 - dados_clientes.csv
 - dicionario.xlsx
 - historico.csv
 - mrr.csv
 - nps_relacional.csv
 - nps_transacional_aquisicao.csv
 - nps_transacional_implantacao.csv
 - nps_transacional_onboarding.csv
 - nps_transacional_produto.csv
 - nps_transacional_suporte.csv
 - telemetria_1.csv
 - telemetria_10.csv
 - telemetria_11.csv
 - telemetria_2.csv
 - telemetria_3.csv
 - telemetria_4.csv
 - telemetria_5.csv
 - telemetria_6.csv
 - telemetria_7.csv
 - telemetria_8.csv
 - telemetria_9.csv
 - tickets.csv


In [16]:
tratamentos = {
    'clientes_desde.csv': tratar_clientes_desde,
    'contratacoes_ultimos_12_meses.csv': tratar_contratacoes_12m,
    'dados_clientes.csv': tratar_dados_clientes,
    'historico.csv': tratar_historico,
    'mrr.csv': tratar_mrr,
    'nps_relacional.csv': tratar_nps_relacional,
    'nps_transacional_aquisicao.csv': tratar_nps_aquisicao,
    'nps_transacional_implantacao.csv': tratar_nps_implantacao,
    'nps_transacional_onboarding.csv': tratar_nps_onboarding,
    'nps_transacional_produto.csv': tratar_nps_produto,
    'nps_transacional_suporte.csv': tratar_nps_suporte,
    'tickets.csv': tratar_tickets,
    'telemetria_1.csv': tratar_telemetria,
    'telemetria_2.csv': tratar_telemetria,
    'telemetria_3.csv': tratar_telemetria,
    'telemetria_4.csv': tratar_telemetria,
    'telemetria_5.csv': tratar_telemetria,
    'telemetria_6.csv': tratar_telemetria,
    'telemetria_7.csv': tratar_telemetria,
    'telemetria_8.csv': tratar_telemetria,
    'telemetria_9.csv': tratar_telemetria,
    'telemetria_10.csv': tratar_telemetria,
    'telemetria_11.csv': tratar_telemetria
}

In [17]:
CONTAINER_PROCESSADOS = "csvs-tratados-gpt"

In [ ]:
for nome_arquivo, funcao in tratamentos.items():
    print(f"🔄 Processando arquivo: {nome_arquivo}")
    blob_client = blob_service_client.get_blob_client(container=CONTAINER_NAME, blob=nome_arquivo)
    downloaded_blob = BytesIO(blob_client.download_blob().readall())
    df_tratado = funcao(downloaded_blob)
    csv_buffer = StringIO()
    df_tratado.to_csv(csv_buffer, index=False, encoding="utf-8-sig")

    # Cria o container se ainda não existir
    #blob_service_client.create_container(container_name) # Comentamos para não ocasionar erro ao tentar criar um container já existente

    # 3. Criamos client do blob no novo container
    blob_client = blob_service_client.get_blob_client(
        container=CONTAINER_PROCESSADOS,
        blob=nome_arquivo
    )

    # 4. Upload do CSV
    blob_client.upload_blob(csv_buffer.getvalue(), overwrite=True)

    print(f"✅ Arquivo {nome_arquivo} exportado para o container 'clusters' com sucesso!")
    print("\n---\n")


🔄 Processando arquivo: clientes_desde.csv
✅ Arquivo clientes_desde tratado
✅ Arquivo clientes_desde.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: contratacoes_ultimos_12_meses.csv
✅ Arquivo contratacoes_12m tratado
✅ Arquivo contratacoes_ultimos_12_meses.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: dados_clientes.csv
✅ Arquivo dados_clientes tratado
✅ Arquivo dados_clientes.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: historico.csv
✅ Arquivo historico tratado
✅ Arquivo historico.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: mrr.csv
✅ Arquivo mrr tratado
✅ Arquivo mrr.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: nps_relacional.csv
✅ Arquivo nps_relacional tratado
✅ Arquivo nps_relacional.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: nps_transacional_aquisicao.

/home/ryan/fiap/TotsVisio/src/processing/tratamento.py:147: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['DATA_RESPOSTA'] = pd.to_datetime(df['DATA_RESPOSTA'], errors='coerce')


✅ Arquivo nps_produto tratado
✅ Arquivo nps_transacional_produto.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: nps_transacional_suporte.csv
✅ Arquivo nps_suporte tratado
✅ Arquivo nps_transacional_suporte.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: tickets.csv
✅ Arquivo tickets tratado
✅ Arquivo tickets.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: telemetria_1.csv
✅ Arquivo telemetria tratado
✅ Arquivo telemetria_1.csv exportado para o container 'clusters' com sucesso!

---

🔄 Processando arquivo: telemetria_2.csv
✅ Arquivo telemetria tratado
✅ Arquivo telemetria_2.csv exportado para o container 'clusters' com sucesso!

---

yeah
